In [30]:
import openai, json

client = openai.OpenAI()

# ============================================================
# 1. Memory structures
# ============================================================
messages = []          # Full conversation history (LLM memory)
user_profile = {       # Structured memory for quick reference
    "favorite_genres": [],
    "watched_movies": [],
}

In [ ]:
# ============================================================
# 2. System prompt — instructs the LLM HOW to use memory
# ============================================================
SYSTEM_PROMPT = """You are a Movie Expert Chatbot that remembers everything the user tells you.

CAPABILITIES:
- Look up currently popular movies
- Get detailed information about any movie by ID
- Find similar movies to any given movie by ID

BEHAVIOR:
- When the user asks about popular/trending movies, call get_popular_movies.
- When the user asks about a specific movie's details, call get_movie_details.
- When the user asks for movies similar to one, call get_similar_movies.
- You may chain multiple tool calls if needed (e.g., get details then find similar).
- Always base your answers on real API data, not your training knowledge.

MEMORY RULES:
- Remember the user's favorite genres across the conversation.
- Remember movies the user has already watched.
- NEVER recommend a movie the user said they already watched.
- Reference what you know about the user to personalize responses.

Be enthusiastic about movies."""

messages.append({"role": "system", "content": SYSTEM_PROMPT})


In [32]:
# ============================================================
# 3. Helper — update structured profile from user message
# ============================================================
def update_user_profile(user_msg: str):
    """
    Use a cheap LLM call to extract genres and watched movies
    from the user's message and update the profile.
    This runs silently — the user never sees it.
    """
    extraction_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "Extract any favorite genres and already-watched movie titles "
                    "from the user message. Return ONLY valid JSON with two keys: "
                    '"genres" (list of strings) and "watched" (list of strings). '
                    "If nothing found, return empty lists. No markdown, no explanation."
                ),
            },
            {"role": "user", "content": user_msg},
        ],
    )

    try:
        extracted = json.loads(extraction_response.choices[0].message.content)
        for genre in extracted.get("genres", []):
            if genre not in user_profile["favorite_genres"]:
                user_profile["favorite_genres"].append(genre)
        for movie in extracted.get("watched", []):
            if movie not in user_profile["watched_movies"]:
                user_profile["watched_movies"].append(movie)
    except (json.JSONDecodeError, AttributeError):
        pass  # Extraction failed silently — main chat still works

In [33]:
# ============================================================
# 4. Build context-aware prompt with current profile state
# ============================================================
def build_profile_reminder():
    """Inject a concise profile summary so the LLM never 'forgets'."""
    if not user_profile["favorite_genres"] and not user_profile["watched_movies"]:
        return None

    parts = []
    if user_profile["favorite_genres"]:
        parts.append(f"Favorite genres: {', '.join(user_profile['favorite_genres'])}")
    if user_profile["watched_movies"]:
        parts.append(f"Already watched: {', '.join(user_profile['watched_movies'])}")

    return (
        "[MEMORY SNAPSHOT — use this to personalize your response]\n"
        + "\n".join(parts)
        + "\nDo NOT recommend any movie listed above as already watched."
    )



In [34]:
# ============================================================
# 5. Core chat function
# ============================================================
def chat(user_input: str):
    # A) Extract profile data from user message
    update_user_profile(user_input)

    # B) Append user message to conversation history
    messages.append({"role": "user", "content": user_input})

    # C) Build messages payload — inject profile reminder as a
    #    system message right before the LLM call so it's fresh
    call_messages = list(messages)  # shallow copy
    reminder = build_profile_reminder()
    if reminder:
        call_messages.append({"role": "system", "content": reminder})

    # D) Call LLM
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=call_messages,
    )

    ai_message = response.choices[0].message.content

    # E) Append assistant response to conversation history
    messages.append({"role": "assistant", "content": ai_message})

    return ai_message


In [35]:
# ============================================================
# 6. Main loop (terminal) — swap this for Jupyter cells below
# ============================================================
if __name__ == "__main__":
    print("=== Movie Recommendation Chatbot (with Memory) ===")
    print('Type your message (or "q" to quit)\n')

    while True:
        user_input = input("User: ")
        if user_input.lower() in ("quit", "q"):
            break

        response = chat(user_input)
        print(f"AI: {response}\n")

    # Show final memory state
    print("\n--- User Profile (Memory) ---")
    print(json.dumps(user_profile, ensure_ascii=False, indent=2))

=== Movie Recommendation Chatbot (with Memory) ===
Type your message (or "q" to quit)

AI: That's awesome! Science fiction is such a fascinating genre with endless possibilities. If you have any favorite sci-fi movies you've already seen, feel free to share them, and I’ll make sure to remember them so I can suggest new ones. Do you have any favorite sci-fi films you want to tell me about?

AI: Got it! You love sci-fi movies, and you've already seen "Interstellar" and "Inceptions." How about exploring some other fantastic sci-fi films that you haven't seen yet? I can suggest some captivating options! Would you like me to?

AI: Since you love sci-fi and haven't mentioned seeing it yet, I recommend you check out "Blade Runner 2049." It's a stunning sequel to the original "Blade Runner" and delves into themes of identity, humanity, and technology. The visuals are breathtaking, and the story is incredibly thought-provoking. I think you'll really enjoy it! Let me know what you think if you d